# Ground Truth Entity Recognition

In [1]:
# %pip install awswrangler
# %pip install bleach
# %pip install nltk

In [2]:
import os
import sys
import nltk
import json
import boto3 
import numpy as np
import pandas as pd
import awswrangler as wr
from bleach.sanitizer import Cleaner

nltk.download('punkt')

from sagemaker import get_execution_role
role = get_execution_role()

sys.path.append(os.path.dirname(os.getcwd()) + '/src')
print('added ', os.path.dirname(os.getcwd()) + '/src', ' to sys.')
from preprocess import preprocess_data

[nltk_data] Downloading package punkt to /home/ec2-user/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


added  /home/ec2-user/SageMaker/ons-poc-manobras-catalog/src  to sys.


[nltk_data] Downloading package punkt to /home/ec2-user/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
preproc = preprocess_data()

Preprocessing module
started at  2021-09-21 14:29:08.051237


In [4]:
# reading file contents from s3 bucket
df = preproc.get_baseline_text()
print(df.info())
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6025 entries, 0 to 6024
Data columns (total 26 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Id                       6025 non-null   int64  
 1   documentUID              6025 non-null   object 
 2   contentUID               6025 non-null   object 
 3   Title                    6023 non-null   object 
 4   Level                    6025 non-null   object 
 5   hierarquia               6025 non-null   object 
 6   relatedEmbedded          0 non-null      float64
 7   relatedLinks             0 non-null      float64
 8   MpoAreaEletrica          6025 non-null   object 
 9   MpoCentro                6025 non-null   object 
 10  MpoEquipamentos          2370 non-null   object 
 11  MpoIdentificador         6025 non-null   object 
 12  MpoLigadoDesligado       6025 non-null   object 
 13  MpoLocalizacaoTaxonomia  6025 non-null   object 
 14  MpoMopsAssociadas       

,Id,documentUID,contentUID,Title,Level,hierarquia,relatedEmbedded,relatedLinks,MpoAreaEletrica,MpoCentro,...,MpoRevisaoReadOnly,MpoSubmodulo,MpoTema,MpoTextoVigenciaPorData,MpoTipoDocumento,nomeDocumento,statusItem,StatusDocumento,texto,Created
0,181505,e401f716-b9c8-44cf-a961-75dd62a2a8aa,cffb9a6b-ba1f-4756-82d4-cd88a5230036,OBJETIVO,1,OBJETIVO,NaN,NaN,2TR,COSR-NCO,...,27,5.12 - Preparação para Manobras na Área 230 kV...,Instruções da Região Norte - Área 230 kV Tramo...,2020-03-11 03:00:00,Preparação para Manobras,Preparação para Manobras na Área 230 kV Tramo ...,Aprovado,Vigente,Estabelecer procedimentos para manobras de equ...,30/07/2021 14:43:17
1,181506,e401f716-b9c8-44cf-a961-75dd62a2a8aa,7b8fcaee-0dce-41f1-b1d9-3dacc9871960,CONSIDERAÇÕES GERAIS,2,CONSIDERAÇÕES GERAIS,NaN,NaN,2TR,COSR-NCO,...,27,5.12 - Preparação para Manobras na Área 230 kV...,Instruções da Região Norte - Área 230 kV Tramo...,2020-03-11 03:00:00,Preparação para Manobras,Preparação para Manobras na Área 230 kV Tramo ...,Aprovado,Vigente,2.1. Os procedimentos contidos nest...,30/07/2021 14:43:18
2,181507,e401f716-b9c8-44cf-a961-75dd62a2a8aa,5a57338c-7b88-466c-a769-51ce87e128b1,PREPARAÇÃO PARA MANOBRAS DE LINHAS DE TRANSMISSÃO,3,PREPARAÇÃO PARA MANOBRAS DE LINHAS DE TRANSMISSÃO,NaN,NaN,2TR,COSR-NCO,...,27,5.12 - Preparação para Manobras na Área 230 kV...,Instruções da Região Norte - Área 230 kV Tramo...,2020-03-11 03:00:00,Preparação para Manobras,Preparação para Manobras na Área 230 kV Tramo ...,Aprovado,Vigente,1. OBJETIVO2....................................,30/07/2021 14:43:18
3,181508,e401f716-b9c8-44cf-a961-75dd62a2a8aa,2d387a90-353d-4411-b263-d528fbba8e7c,LT 230 KV Tucuruí / Altamira,3.1,3 - PREPARAÇÃO PARA MANOBRAS DE LINHAS DE TRAN...,NaN,NaN,2TR,COSR-NCO,...,27,5.12 - Preparação para Manobras na Área 230 kV...,Instruções da Região Norte - Área 230 kV Tramo...,2020-03-11 03:00:00,Preparação para Manobras,Preparação para Manobras na Área 230 kV Tramo ...,Aprovado,Vigente,3.1.1. Desenergização da LT 230 kV Tucur...,30/07/2021 14:43:18
4,181509,e401f716-b9c8-44cf-a961-75dd62a2a8aa,0ba555c0-1b6a-4d16-8f76-8758b214042c,LT 230 KV Altamira / Xingu,3.2,3 - PREPARAÇÃO PARA MANOBRAS DE LINHAS DE TRAN...,NaN,NaN,2TR,COSR-NCO,...,27,5.12 - Preparação para Manobras na Área 230 kV...,Instruções da Região Norte - Área 230 kV Tramo...,2020-03-11 03:00:00,Preparação para Manobras,Preparação para Manobras na Área 230 kV Tramo ...,Aprovado,Vigente,3.2.1. Desenergização da LT 230 kV Altam...,30/07/2021 14:43:18


In [5]:
df = df[df['MpoEquipamentos'].isnull()==False]
df.head()

,Id,documentUID,contentUID,Title,Level,hierarquia,relatedEmbedded,relatedLinks,MpoAreaEletrica,MpoCentro,...,MpoRevisaoReadOnly,MpoSubmodulo,MpoTema,MpoTextoVigenciaPorData,MpoTipoDocumento,nomeDocumento,statusItem,StatusDocumento,texto,Created
3,181508,e401f716-b9c8-44cf-a961-75dd62a2a8aa,2d387a90-353d-4411-b263-d528fbba8e7c,LT 230 KV Tucuruí / Altamira,3.1,3 - PREPARAÇÃO PARA MANOBRAS DE LINHAS DE TRAN...,NaN,NaN,2TR,COSR-NCO,...,27,5.12 - Preparação para Manobras na Área 230 kV...,Instruções da Região Norte - Área 230 kV Tramo...,2020-03-11 03:00:00,Preparação para Manobras,Preparação para Manobras na Área 230 kV Tramo ...,Aprovado,Vigente,3.1.1. Desenergização da LT 230 kV Tucur...,30/07/2021 14:43:18
4,181509,e401f716-b9c8-44cf-a961-75dd62a2a8aa,0ba555c0-1b6a-4d16-8f76-8758b214042c,LT 230 KV Altamira / Xingu,3.2,3 - PREPARAÇÃO PARA MANOBRAS DE LINHAS DE TRAN...,NaN,NaN,2TR,COSR-NCO,...,27,5.12 - Preparação para Manobras na Área 230 kV...,Instruções da Região Norte - Área 230 kV Tramo...,2020-03-11 03:00:00,Preparação para Manobras,Preparação para Manobras na Área 230 kV Tramo ...,Aprovado,Vigente,3.2.1. Desenergização da LT 230 kV Altam...,30/07/2021 14:43:18
5,181510,e401f716-b9c8-44cf-a961-75dd62a2a8aa,63deb90e-ec9d-4ec1-b93c-745b0554fec8,LT 230 KV Altamira / Transamazônica C1 ou C2,3.3,3 - PREPARAÇÃO PARA MANOBRAS DE LINHAS DE TRAN...,NaN,NaN,2TR,COSR-NCO,...,27,5.12 - Preparação para Manobras na Área 230 kV...,Instruções da Região Norte - Área 230 kV Tramo...,2020-03-11 03:00:00,Preparação para Manobras,Preparação para Manobras na Área 230 kV Tramo ...,Aprovado,Vigente,3.3.1. Desenergização da LT 230 kV Altam...,30/07/2021 14:43:19
6,181511,e401f716-b9c8-44cf-a961-75dd62a2a8aa,9f861169-0b28-4673-8ffe-5f0ef3a30281,LT 230 KV Transamazônica / Rurópolis,3.4,3 - PREPARAÇÃO PARA MANOBRAS DE LINHAS DE TRAN...,NaN,NaN,2TR,COSR-NCO,...,27,5.12 - Preparação para Manobras na Área 230 kV...,Instruções da Região Norte - Área 230 kV Tramo...,2020-03-11 03:00:00,Preparação para Manobras,Preparação para Manobras na Área 230 kV Tramo ...,Aprovado,Vigente,3.4.1. Desenergização da LT 230 kV Trans...,30/07/2021 14:43:19
7,181512,e401f716-b9c8-44cf-a961-75dd62a2a8aa,4f612ba0-e957-415c-bb37-29be4c1a5896,LT 230 KV Tapajós II / Transamazônica,3.5,3 - PREPARAÇÃO PARA MANOBRAS DE LINHAS DE TRAN...,NaN,NaN,2TR,COSR-NCO,...,27,5.12 - Preparação para Manobras na Área 230 kV...,Instruções da Região Norte - Área 230 kV Tramo...,2020-03-11 03:00:00,Preparação para Manobras,Preparação para Manobras na Área 230 kV Tramo ...,Aprovado,Vigente,3.5.1. Desenergização da LT 230 kV Tapaj...,30/07/2021 14:43:19


In [6]:
df['MpoTema'].value_counts()

Instruções da Região Sul                                           709
Preparação para Manobras                                           575
Instruções da Região Sudeste/Centro Oeste - Área São Paulo         253
3.1.3. Preparação para Manobras                                    205
Instruções da Região Sudedeste/Centro Oeste - Área Minas Gerais    123
Instruções da Região Sudeste/Centro Oeste - Área Rio               117
Instruções da Região NE - Área Norte                               113
Instruções da Região NE - Área Sudoeste                             90
Interligação entre Regiões                                          43
Instruções da Região Sudeste/Centro Oeste - Área Rio Grande         26
Instruções da Região Norte - Área 500/230 kV Pará                   23
Instruções da Região Norte - Área 230 kV Tramo Oeste                12
Instruções da Região NE - Geral                                      7
Name: MpoTema, dtype: int64

In [7]:
df_list = preproc.separate_df_by_region(df, column='MpoTema')

In [8]:
for sublist in range(len(df_list)):
    print(df_list[sublist][0], '- amostras :', len(df_list[sublist][1]))

SUL - amostras : 709
SUDESTE - amostras : 519
GERAL - amostras : 897
NORTE - amostras : 35
NORDESTE - amostras : 210


In [9]:
bucket = 'ons-dl-dev-landing'
file_prefix = 'ons/mpo_ds_nlp/regioes/'

for sublist in range(len(df_list)):
    file = file_prefix + df_list[sublist][0].lower() + '.csv'
    obj_uri = 's3://{}/{}'.format(bucket, file)
    df_list[sublist][1].to_csv(obj_uri)
    print(df_list[sublist][0], ' -> ', obj_uri)

SUL  ->  s3://ons-dl-dev-landing/ons/mpo_ds_nlp/regioes/sul.csv
SUDESTE  ->  s3://ons-dl-dev-landing/ons/mpo_ds_nlp/regioes/sudeste.csv
GERAL  ->  s3://ons-dl-dev-landing/ons/mpo_ds_nlp/regioes/geral.csv
NORTE  ->  s3://ons-dl-dev-landing/ons/mpo_ds_nlp/regioes/norte.csv
NORDESTE  ->  s3://ons-dl-dev-landing/ons/mpo_ds_nlp/regioes/nordeste.csv
